In [1]:
import gzip
import numpy as np
import pandas as pd
import scipy.optimize
import random
from collections import defaultdict
import nltk
from sklearn import svm
import string
from sklearn import linear_model
import pickle

In [2]:
# read data
import pickle
data = pickle.load(open("book_data.p", "rb"))
np.random.shuffle(data)

In [10]:
books = {}
book_descriptions = {}
for a,b in data:
    if a['book_id'] not in books:
        books[a['book_id']] = []
        book_descriptions[a['book_id']] = []
    book_descriptions[a['book_id']].append(a['description'])
    books[a['book_id']].append(categoryId[b])

In [11]:
def binary(vec, i):
    v = []
    for item in vec:
        if i in item:
            v.append(1)
        else:
            v.append(0)
    return v

In [12]:
X = []
categories = []
for book_id in books:
    X.append(book_descriptions[book_id])
    categories.append(books[book_id])

In [9]:
reviews = [d['description'] for d,b in data]
categoryId = {}
categoryId['child'] = 0
categoryId['graphic'] = 1
categoryId['fantasy'] = 2
categoryId['history'] = 3
categoryId['poetry'] = 4
categoryId['thriller'] = 5
categoryId['romance'] = 6
categoryId['ya'] = 7
idCategory = {}
for a in categoryId:
    idCategory[categoryId[a]] = a
y = [categoryId[b] for d,b in data]

In [21]:
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
np.random.seed(7)

In [12]:
wordCount = defaultdict(int)
idf = defaultdict(int)
punctuation = set(string.punctuation)
avglen = 0
for d,b in data:
    r = ''.join([c for c in d['description'].lower() if not c in punctuation])
    avglen += len(r.split())
    seen = []
    for w in r.split():
        wordCount[w] += 1
        if w not in seen:
            idf[w] += 1
            seen.append(w)
counts = [(wordCount[w], w) for w in wordCount]
counts.sort()
counts.reverse()
avglen /= len(data)

In [27]:
words = [x[1] for x in counts[:2000]]
wordId = dict(zip(words, range(0,len(words))))
wordSet = set(words)

In [8]:
# read in glove model
def loadGloveModel(gloveFile):
    f = open(gloveFile,'r', encoding='utf-8')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("Done.",len(model)," words loaded!")
    return model
glove = loadGloveModel("C:/Users/liqhtninq/OneDrive/LIGN 167/glove50d.txt")

Done. 400000  words loaded!


In [13]:
X = []
punctuation = string.punctuation
for d,b in data:
    r = ''.join([c for c in d['description'].lower() if not c in punctuation])
    split = r.split()
    vec = []
    for w in split:
        if w in glove:
            vec.append(glove[w])
    X.append(vec)

In [19]:
X_train = X[:160000]
y_train = categories[:160000]
y_test = categories[160000:]
X_test = X[160000:]

In [57]:
data[1]

({'asin': ' ',
  'authors': [{'author_id': '34049', 'role': ''}],
  'average_rating': '4.37',
  'book_id': '11922016',
  'country_code': 'US',
  'description': "They are outcasts. Hal, Stig, and the others - they are the boys the others want no part of. Skandians, as any reader of Ranger's Apprentice could tell you, are known for their size and strength. Not these boys. Yet that doesn't mean they don't have skills. And courage - which they will need every ounce of to do battle at sea against the other bands, the Wolves and the Sharks, in the ultimate race. The icy waters make for a treacherous playing field . . . especially when not everyone thinks of it as playing.<br /><br />John Flanagan, author of the international phenomenon Ranger's Apprentice, creates a new cast of characters to populate his world of Skandians and Araluens, a world millions of young readers around the world have come to know and admire. Full of seafaring adventures and epic battles, Book 1 of The Brotherband Chr

In [60]:
from sklearn import linear_model
clf = linear_model.LogisticRegression()
yb = binary(y_train,6)
clf.fit(X_train, yb)
clf.score(X_test, binary(y_test,6))

0.93950578385920691

In [15]:
# yvec = []
# for d in y:
#     z = np.zeros(5)
#     z[d] = 1
#     yvec.append(z)
from keras.utils import np_utils
# yvec = np_utils.to_categorical(y_test)


Using TensorFlow backend.


In [25]:
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
top_words = 5001
max_review_length = 100
y_train = np_utils.to_categorical(binary(y_train,0))
y_test = np_utils.to_categorical((binary(y_test,0)))
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
print(len(y_test))
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)
# create the model
embedding_vecor_length = 32
model = Sequential()
# model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=128, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(2, activation='linear'))
model.compile(loss='hinge', optimizer='adadelta', metrics=['accuracy'])
# print(model.summary())
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=3, batch_size=64)

22217


ValueError: Input arrays should have the same number of samples as target arrays. Found 40000 input samples and 22217 target samples.

In [ ]:
from keras.models import load_model
model.save('assignment2_model.h5')  # creates a HDF5 file 'my_model.h5'